In [1]:
# Install the necessary packages
%pip install pyarrow
%pip install azure-storage-blob

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
%pip install python-dotenv

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
# Import Necessary Libraries
import pandas as pd
import os
import io
from azure.storage.blob import BlobServiceClient, BlobClient
from dotenv import load_dotenv 

In [4]:
# Extraction Layer
lemco_df = pd.read_csv(r'lemco_logistics_data.csv')

In [5]:
lemco_df.head()

,Transaction_ID,Date,Customer_ID,Product_ID,Quantity,Unit_Price,Total_Cost,Discount_Rate,Sales_Channel,Order_Priority,...,Return_Reason,Payment_Type,Taxable,Region,Country,Customer_Name,Customer_Phone,Customer_Email,Customer_Address,Product_List_Title
0,200,32:25.9,1086,536,3,120.436821,8265.374549,0.20,Online,High,...,Wrong Item,Wire Transfer,False,West,Canada,Customer 200,-10529,customer.200.78@example.com,"275 Second St, Phoenix, USA",Product 53
1,321,55:08.1,1078,523,6,475.724994,4047.850479,NaN,Reseller,Critical,...,Damaged,PayPal,True,South,Mexico,Customer 321,-6256,customer.321.13@sample.com,"478 Third St, New York, USA",Product 33
2,989,12:58.4,1077,535,3,146.400556,NaN,0.05,Direct,Critical,...,Damaged,PayPal,True,West,Canada,Customer 989,-10559,customer.989.99@example.com,"843 Second St, Phoenix, USA",Product 6
3,682,03:14.6,1027,546,6,19.373194,8194.281993,NaN,Reseller,Medium,...,Wrong Item,Cash,True,South,Canada,Customer 682,-6897,customer.682.66@demo.com,"153 Main St, Phoenix, USA",Product 68
4,484,08:06.5,1052,556,8,193.221313,8331.329249,0.20,Direct,Low,...,Late,Cash,False,South,Mexico,Customer 484,-10454,customer.484.3@sample.com,"264 Second St, New York, USA",Product 89


In [6]:
lemco_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Transaction_ID         1005 non-null   int64  
 1   Date                   1005 non-null   object 
 2   Customer_ID            1005 non-null   int64  
 3   Product_ID             1005 non-null   int64  
 4   Quantity               1005 non-null   int64  
 5   Unit_Price             904 non-null    float64
 6   Total_Cost             905 non-null    float64
 7   Discount_Rate          714 non-null    float64
 8   Sales_Channel          1005 non-null   object 
 9   Order_Priority         1005 non-null   object 
 10  Warehouse_Code         1005 non-null   object 
 11  Ship_Mode              1005 non-null   object 
 12  Delivery_Status        1005 non-null   object 
 13  Customer_Satisfaction  1005 non-null   object 
 14  Item_Returned          1005 non-null   bool   
 15  Retu

In [7]:
lemco_df.columns

Index(['Transaction_ID', 'Date', 'Customer_ID', 'Product_ID', 'Quantity',
       'Unit_Price', 'Total_Cost', 'Discount_Rate', 'Sales_Channel',
       'Order_Priority', 'Warehouse_Code', 'Ship_Mode', 'Delivery_Status',
       'Customer_Satisfaction', 'Item_Returned', 'Return_Reason',
       'Payment_Type', 'Taxable', 'Region', 'Country', 'Customer_Name',
       'Customer_Phone', 'Customer_Email', 'Customer_Address',
       'Product_List_Title'],
      dtype='object')

In [9]:
# Data Cleaning and Transformation Layer
lemco_df.fillna({
    'Unit_Price': lemco_df['Unit_Price'].mean(),
    'Total_Cost': lemco_df['Total_Cost'].mean(),
    'Discount_Rate': 0.0,
    'Return_Reason': 'unknown'
}, inplace=True)


In [10]:
lemco_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1005 entries, 0 to 1004
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Transaction_ID         1005 non-null   int64  
 1   Date                   1005 non-null   object 
 2   Customer_ID            1005 non-null   int64  
 3   Product_ID             1005 non-null   int64  
 4   Quantity               1005 non-null   int64  
 5   Unit_Price             1005 non-null   float64
 6   Total_Cost             1005 non-null   float64
 7   Discount_Rate          1005 non-null   float64
 8   Sales_Channel          1005 non-null   object 
 9   Order_Priority         1005 non-null   object 
 10  Warehouse_Code         1005 non-null   object 
 11  Ship_Mode              1005 non-null   object 
 12  Delivery_Status        1005 non-null   object 
 13  Customer_Satisfaction  1005 non-null   object 
 14  Item_Returned          1005 non-null   bool   
 15  Retu

In [11]:
# Remove rows where 'Date' is missing or empty
lemco_df = lemco_df[lemco_df['Date'].notna() & (lemco_df['Date'] != '')]


In [12]:
lemco_df['Date'] = pd.to_datetime(lemco_df['Date'], errors='coerce')
# Optionally, drop rows where 'Date' could not be parsed
lemco_df = lemco_df[lemco_df['Date'].notna()]


C:\Users\Administrator\AppData\Local\Temp\ipykernel_9532\3164789882.py:1: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  lemco_df['Date'] = pd.to_datetime(lemco_df['Date'], errors='coerce')


In [13]:
lemco_df['Date'] = pd.to_datetime(lemco_df['Date'],)

In [16]:
lemco_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 438 entries, 2 to 1004
Data columns (total 25 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Transaction_ID         438 non-null    int64         
 1   Date                   438 non-null    datetime64[ns]
 2   Customer_ID            438 non-null    int64         
 3   Product_ID             438 non-null    int64         
 4   Quantity               438 non-null    int64         
 5   Unit_Price             438 non-null    float64       
 6   Total_Cost             438 non-null    float64       
 7   Discount_Rate          438 non-null    float64       
 8   Sales_Channel          438 non-null    object        
 9   Order_Priority         438 non-null    object        
 10  Warehouse_Code         438 non-null    object        
 11  Ship_Mode              438 non-null    object        
 12  Delivery_Status        438 non-null    object        
 13  Customer_

In [17]:
lemco_df.head()

,Transaction_ID,Date,Customer_ID,Product_ID,Quantity,Unit_Price,Total_Cost,Discount_Rate,Sales_Channel,Order_Priority,...,Return_Reason,Payment_Type,Taxable,Region,Country,Customer_Name,Customer_Phone,Customer_Email,Customer_Address,Product_List_Title
2,989,2025-10-25 12:58:24,1077,535,3,146.400556,5096.553818,0.05,Direct,Critical,...,Damaged,PayPal,True,West,Canada,Customer 989,-10559,customer.989.99@example.com,"843 Second St, Phoenix, USA",Product 6
3,682,2025-10-25 03:14:36,1027,546,6,19.373194,8194.281993,0.00,Reseller,Medium,...,Wrong Item,Cash,True,South,Canada,Customer 682,-6897,customer.682.66@demo.com,"153 Main St, Phoenix, USA",Product 68
4,484,2025-10-25 08:06:30,1052,556,8,193.221313,8331.329249,0.20,Direct,Low,...,Late,Cash,False,South,Mexico,Customer 484,-10454,customer.484.3@sample.com,"264 Second St, New York, USA",Product 89
5,483,2025-10-25 11:21:06,1074,549,8,37.726972,6894.375244,0.05,Reseller,Medium,...,Not Specified,PayPal,False,South,USA,Customer 483,-4913,customer.483.31@example.com,"629 Third St, Los Angeles, USA",Product 27
6,757,2025-10-25 11:21:06,1061,547,7,193.979775,4611.329101,0.15,Direct,Critical,...,Late,Credit Card,True,South,Canada,Customer 757,-5788,customer.757.80@demo.com,"598 Main St, Los Angeles, USA",Product 83


In [19]:
# Customer Table
customer = lemco_df[['Customer_ID', 'Customer_Name', 'Customer_Phone', 'Customer_Email', 'Customer_Address']].copy().drop_duplicates().reset_index(drop=True)

customer.head()


,Customer_ID,Customer_Name,Customer_Phone,Customer_Email,Customer_Address
0,1077,Customer 989,-10559,customer.989.99@example.com,"843 Second St, Phoenix, USA"
1,1027,Customer 682,-6897,customer.682.66@demo.com,"153 Main St, Phoenix, USA"
2,1052,Customer 484,-10454,customer.484.3@sample.com,"264 Second St, New York, USA"
3,1074,Customer 483,-4913,customer.483.31@example.com,"629 Third St, Los Angeles, USA"
4,1061,Customer 757,-5788,customer.757.80@demo.com,"598 Main St, Los Angeles, USA"


In [20]:
customer.shape

(437, 5)

In [21]:
# Products Table
products = lemco_df[['Product_ID', 'Quantity', 'Unit_Price', 'Total_Cost', 'Discount_Rate','Product_List_Title']].copy().drop_duplicates().reset_index(drop=True)

products.head()

,Product_ID,Quantity,Unit_Price,Total_Cost,Discount_Rate,Product_List_Title
0,535,3,146.400556,5096.553818,0.05,Product 6
1,546,6,19.373194,8194.281993,0.00,Product 68
2,556,8,193.221313,8331.329249,0.20,Product 89
3,549,8,37.726972,6894.375244,0.05,Product 27
4,547,7,193.979775,4611.329101,0.15,Product 83


In [22]:
# Transactions_Fact_ Table

transaction_fact = lemco_df.merge(customer, on=['Customer_ID','Customer_Name','Customer_Phone','Customer_Email', 'Customer_Address'], how='left') \
                     .merge(products, on=['Product_ID', 'Quantity','Product_List_Title','Unit_Price','Total_Cost', 'Discount_Rate'], how='left') \
                     [['Transaction_ID', 'Date', 'Customer_ID', 'Product_ID','Sales_Channel','Order_Priority', \
                         'Warehouse_Code', 'Ship_Mode', 'Delivery_Status','Customer_Satisfaction', 'Item_Returned', 'Return_Reason', \
                         'Payment_Type', 'Taxable', 'Region', 'Country']]
                     
transaction_fact.head()

,Transaction_ID,Date,Customer_ID,Product_ID,Sales_Channel,Order_Priority,Warehouse_Code,Ship_Mode,Delivery_Status,Customer_Satisfaction,Item_Returned,Return_Reason,Payment_Type,Taxable,Region,Country
0,989,2025-10-25 12:58:24,1077,535,Direct,Critical,WH-1,Overnight,Pending,Unsatisfied,True,Damaged,PayPal,True,West,Canada
1,682,2025-10-25 03:14:36,1027,546,Reseller,Medium,WH-1,Express,Pending,Unsatisfied,False,Wrong Item,Cash,True,South,Canada
2,484,2025-10-25 08:06:30,1052,556,Direct,Low,WH-2,2-Day,Delivered,Satisfied,True,Late,Cash,False,South,Mexico
3,483,2025-10-25 11:21:06,1074,549,Reseller,Medium,WH-3,2-Day,Backorder,Unsatisfied,True,Not Specified,PayPal,False,South,USA
4,757,2025-10-25 11:21:06,1061,547,Direct,Critical,WH-3,Express,Cancelled,Neutral,False,Late,Credit Card,True,South,Canada


In [23]:
transaction_fact.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 438 entries, 0 to 437
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype         
---  ------                 --------------  -----         
 0   Transaction_ID         438 non-null    int64         
 1   Date                   438 non-null    datetime64[ns]
 2   Customer_ID            438 non-null    int64         
 3   Product_ID             438 non-null    int64         
 4   Sales_Channel          438 non-null    object        
 5   Order_Priority         438 non-null    object        
 6   Warehouse_Code         438 non-null    object        
 7   Ship_Mode              438 non-null    object        
 8   Delivery_Status        438 non-null    object        
 9   Customer_Satisfaction  438 non-null    object        
 10  Item_Returned          438 non-null    bool          
 11  Return_Reason          438 non-null    object        
 12  Payment_Type           438 non-null    object        
 13  Taxab

In [24]:
transaction_fact['Date'] = transaction_fact['Date'].astype('datetime64[ns]')

In [25]:
# Temprary Loading
customer.to_csv(r'dataset/customer.csv', index=False)
products.to_csv(r'dataset/products.csv', index=False)
transaction_fact.to_csv(r'dataset/transaction_fact.csv', index=False)

In [26]:
# Data Loading
# Azure blob connection
load_dotenv()
connect_str = os.getenv('CONNECT_STR')
blob_service_client = BlobServiceClient.from_connection_string(connect_str)

container_name = os.getenv('CONTAINER_NAME')
container_client = blob_service_client.get_container_client(container_name)

In [27]:
# Create a function to upload files to Azure Blob Storage as a parquet file
def upload_df_to_blob_as_parquet(df, container_client, blob_name):
    buffer = io.BytesIO()
    df.to_parquet(buffer, index=False)
    buffer.seek(0)
    blob_client = container_client.get_blob_client(blob_name)
    blob_client.upload_blob(buffer, blob_type="BlockBlob", overwrite=True)
    print(f"{blob_name} uploaded to Azure Blob Storage successfully.")

In [28]:
upload_df_to_blob_as_parquet(customer, container_client, 'rawdata/customer.parquet')
upload_df_to_blob_as_parquet(products, container_client, 'rawdata/products.parquet')
upload_df_to_blob_as_parquet(transaction_fact, container_client, 'rawdata/transaction_fact.parquet')


rawdata/customer.parquet uploaded to Azure Blob Storage successfully.
rawdata/products.parquet uploaded to Azure Blob Storage successfully.
rawdata/transaction_fact.parquet uploaded to Azure Blob Storage successfully.
